# Data Preprocessing:
1. Import necessary libraries
2. Read dataset
3. Sanity check of data
4. Missing value treatments
5. Duplicates & garbage value treatments
6. Outliers treatment
7. Exploratory data analysis (EDA)
8. Encoding of data
9. Normalisation/ Standardization

# Importing Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Reading dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv("/content/drive/MyDrive/city_hour_transformed.csv")
df.head()

Mounted at /content/drive


,Unnamed: 0,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,...,State,Status,Region,Day_period,Month,Year,Season,Weekday_or_weekend,Regular_day_or_holiday,AQ_Acceptability
0,0,Ahmedabad,2015-01-01 01:00:00,NaN,NaN,1.00,40.01,36.37,NaN,1.00,...,Gujarat,Active,5. Western,4. Night,01. Jan,2015,1. Winter,Weekday,Regular day,Unacceptable
1,1,Ahmedabad,2015-01-01 02:00:00,NaN,NaN,0.02,27.75,19.73,NaN,0.02,...,Gujarat,Active,5. Western,4. Night,01. Jan,2015,1. Winter,Weekday,Regular day,Unacceptable
2,2,Ahmedabad,2015-01-01 03:00:00,NaN,NaN,0.08,19.32,11.08,NaN,0.08,...,Gujarat,Active,5. Western,4. Night,01. Jan,2015,1. Winter,Weekday,Regular day,Unacceptable
3,3,Ahmedabad,2015-01-01 04:00:00,NaN,NaN,0.30,16.45,9.20,NaN,0.30,...,Gujarat,Active,5. Western,1. Morning,01. Jan,2015,1. Winter,Weekday,Regular day,Unacceptable
4,4,Ahmedabad,2015-01-01 05:00:00,NaN,NaN,0.12,14.90,7.85,NaN,0.12,...,Gujarat,Active,5. Western,1. Morning,01. Jan,2015,1. Winter,Weekday,Regular day,Unacceptable


#Drop Xylene column

In [ ]:
df.drop(columns=['Xylene'], inplace=True)
df.shape

(4476937, 28)

# Check null columns

In [ ]:
df.isnull().sum()

Unnamed: 0                      0
City                            0
Datetime                        0
PM2.5                      600906
PM10                      1413338
NO                         496494
NO2                        497313
NOx                        390824
NH3                       1231007
CO                         255065
SO2                        623721
O3                         627541
Benzene                    484318
Toluene                    695586
AQI                        577120
AQI_Bucket                      0
StationId                       0
StationName                     0
State                           0
Status                          0
Region                          0
Day_period                      0
Month                           0
Year                            0
Season                          0
Weekday_or_weekend              0
Regular_day_or_holiday          0
AQ_Acceptability                0
dtype: int64

#Data preprocessing using Simple Imputer for PM10 and NH3

In [ ]:
from sklearn.impute import SimpleImputer
# Select the columns to impute
columns_to_impute = ['PM10', 'NH3']
# Initialize the SimpleImputer for mean imputation
mean_imputer = SimpleImputer(strategy='mean')
# Fit the imputer on the selected columns and transform them
df[columns_to_impute] = mean_imputer.fit_transform(df[columns_to_impute])
# Check for any remaining missing values
print(df.isnull().sum())
# Display the first few rows of the DataFrame
df.head()


# Imputer other columns with mean

In [ ]:
numeric_columns = ['PM2.5', 'NO', 'NO2', 'NOx', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene',]
for col in numeric_columns:
    if df[col].isnull().sum() / len(df) < 0.2:  # Check if missing percentage is below 20%
        mean_val = df[col].mean()  # Calculate mean
        df[col].fillna(mean_val, inplace=True)

In [ ]:
df.isnull().sum()

# Drop rows where AQI is missing

In [ ]:
df = df.dropna(subset=['AQI'])
print(df.isnull().sum())
df.shape

# Detect duplicate records

In [ ]:
duplicates = df.duplicated()
print(duplicates)

# Summary

In [ ]:
summary_statistics = df.describe()
print(summary_statistics)

#Range-Based Outlier Removal and Treatment

In [ ]:
# Define expected ranges
expected_ranges = {
    'PM2.5': (0, 500),
    'PM10': (0, 500),
    'NO': (0, 500),
    'NO2': (0, 500),
    'NOx': (0, 500),
    'NH3': (0, 500),
    'CO': (0, 50),
    'SO2': (0, 500),
    'O3': (0, 500),
    'Benzene': (0, 100),
    'Toluene': (0, 100),
    'AQI': (0, 500)
}

# Initialize a list to store indices of rows with out-of-range values
rows_to_drop = []

# Check if values are within expected ranges and collect indices of out-of-range rows
for column, (min_val, max_val) in expected_ranges.items():
    out_of_range_indices = df[~df[column].between(min_val, max_val)].index
    if not out_of_range_indices.empty:
        rows_to_drop.extend(out_of_range_indices)

# Remove duplicate indices (if any)
rows_to_drop = list(set(rows_to_drop))

# Drop rows with out-of-range values
df= df.drop(rows_to_drop)

# Print the shape of the cleaned DataFrame
print("Shape of cleaned DataFrame:", df.shape)

# OLS Regression testing

In [ ]:
import statsmodels.api as sm
features = ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene']
target = 'AQI'

# Create the feature matrix (X) and target vector (y)
X = df[features]
y = df[target]

# Add a constant to the feature matrix (for the intercept term)
X = sm.add_constant(X)

# Step 5: Perform OLS Regression
# Fit the OLS model
model = sm.OLS(y, X).fit()

# Get the summary of the regression
summary = model.summary()
print(summary)

# Outliers detection
###Z-score Calculation:

The Z-score of a data point is calculated using the formula:
𝑍 = ( 𝑋 − 𝜇 )/ 𝜎 ​


Where :
*   𝑋 is the data point,
*   𝜇 is the mean of the data, and
*   𝜎 is the standard deviation of the data.


Once the Z-scores are calculated for each data point in a column, the absolute value of each Z-score is compared to the threshold.

Commonly used thresholds are 3 and -3, meaning data points with Z-scores greater than 3 or less than -3 are considered outliers.

If the absolute value of a Z-score exceeds the threshold, the corresponding data point is flagged as an outlier.

For example, if the threshold is 5, any data point with a Z-score less than -5 or greater than 5 is considered an outlier.

In [ ]:
from scipy.stats import zscore
# Select only numeric columns for outlier detection
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Calculate Z-scores for each column separately to avoid indexing issues
z_scores = numeric_df.apply(zscore)

# Define a threshold for Z-scores to identify outliers (commonly 3 or -3)
threshold = 5

# Identify outliers in each numeric column
outliers_count = {}
for column in numeric_df.columns:
    outliers = np.where(np.abs(z_scores[column]) > threshold)[0]
    outliers_count[column] = len(outliers)

# Display the number of outliers in each numeric column
print("Number of outliers in each numeric column:")
for column, count in outliers_count.items():
    print(f"{column}: {count}")


# Outliers treatment

In [ ]:
# Select only numeric columns
numeric_columns = df.select_dtypes(include=[np.number]).columns

# Calculate Z-scores
z_scores = np.abs((df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std())

# Define a threshold for outliers
threshold = 5

# Identify rows with any outlier
outliers = (z_scores > threshold).any(axis=1)
num_outliers = outliers.sum()

print("Number of records that would be removed as outliers:", num_outliers)
# Filter out rows with outliers
df = df[~outliers]

# Display the shape of the cleaned DataFrame
print("Shape of the cleaned DataFrame after removing outliers:", df.shape)


# EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot histograms for each column
df.hist(bins=50, figsize=(20, 15))
plt.show()


# Heatmap for finding correlation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Select only numeric columns from the DataFrame
numeric_columns = df.select_dtypes(include=['float64', 'int64'])

# Correlation matrix
correlation_matrix = numeric_columns.corr()

# Heatmap of the correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()


# All categorical columns

In [ ]:
# Select columns with object dtype (assuming categorical columns are of type 'object')
categorical_columns = df.select_dtypes(include=['object']).columns

# Print the list of categorical columns
print("Categorical columns:")
print(categorical_columns)

# Count the number of categorical columns
num_categorical_columns = len(categorical_columns)
print("Number of categorical columns:", num_categorical_columns)


# Finding cardinality (Different values for a particular column)

In [ ]:
# List of categorical columns
categorical_columns = ['City', 'Datetime', 'AQI_Bucket', 'StationId', 'StationName', 'State',
                       'Status', 'Region', 'Day_period', 'Month', 'Season',
                       'Weekday_or_weekend', 'Regular_day_or_holiday', 'AQ_Acceptability']
# Calculate cardinality for each categorical column
cardinality = {}
for column in categorical_columns:
    cardinality[column] = df[column].nunique()

# Display cardinality of each categorical column
for column, value in cardinality.items():
    print(f"{column}: {value}")

# Data encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# Assume df is your original DataFrame

# Define columns to encode
columns_to_encode = ['Region', 'Day_period', 'Season', 'Weekday_or_weekend', 'Regular_day_or_holiday']

# One-hot encode categorical columns except for 'Month'
encoder = OneHotEncoder(drop='first', sparse=False)
encoded_data = encoder.fit_transform(df[columns_to_encode])

# Convert the encoded data into a DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_encode))

# Label encode 'Month'
label_encoder = LabelEncoder()
df['Month_encoded'] = label_encoder.fit_transform(df['Month'])

# Drop the original columns that were encoded and the original 'Month' column
df_temp = df.drop(columns=columns_to_encode + ['Month'])

# Concatenate the encoded DataFrame with the remaining original DataFrame
df_encoded = pd.concat([df_temp, encoded_df], axis=1)

# Display the encoded DataFrame
print(df_encoded.head())


# Heatmap to visualize the relationship of all encoded columns with AQI

In [ ]:
# Select only numeric columns from the DataFrame
numeric_columns = df_encoded.select_dtypes(include=['float64', 'int64'])

# Correlation matrix
correlation_matrix = numeric_columns.corr()

# Heatmap of the correlation matrix
plt.figure(figsize=(25, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()
print(df.columns)

# Reset index and rename index column

In [ ]:
# Drop the current 'Serial number' column if it exists
df.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')

# Reset the index of the DataFrame
df.reset_index(drop=True, inplace=True)

# Create a new 'Serial number' column starting from 1
df['Serial number'] = df.index + 1

# Reorder the columns to move 'Serial number' to the first position
columns_order = ['Serial number'] + [col for col in df.columns if col != 'Serial number']
df = df[columns_order]

# Display the first few rows to verify
df.head()


# Set all decimal upto 2 places


In [ ]:
# Define the number of decimal places
decimal_places = 2

# Select only numerical columns
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Round numerical columns to the specified number of decimal places
df[numeric_columns] = df[numeric_columns].round(decimal_places)

# Display the first few rows of the modified DataFrame
print(df.head())
df.shape

# Download CSV before standardization


In [ ]:
# # Save the dataframe to a CSV file
# df.to_csv('processed_data1.csv', index=False)

# # Download the file
# from google.colab import files
# files.download('processed_data1.csv')

# Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

# Select numeric columns for normalization
numeric_columns = ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene',
                   'Toluene', 'AQI']

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the data
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# Display the first few rows of the normalized data
print(df.head())

# Download CSV after Standardization

In [ ]:
# Save the dataframe to a CSV file
df.to_csv('processed_data2.csv', index=False)

# Download the file
from google.colab import files
files.download('processed_data2.csv')